In [ ]:
pip install --quiet autogen langchain langchain_openai langchain-community langchain_experimental gradio psycopg2 python-dotenv

In [ ]:
import autogen
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_experimental.sql.base import SQLDatabaseChain
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
import gradio as gr
import asyncio
import psycopg2

In [ ]:
# Load environment variables from the .env file from the same directory as notebook 
load_dotenv()

# Retrieve environment variables
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_DB = os.getenv('POSTGRES_DB')
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')

In [ ]:
# Construct the database URI
shipment_db_uri = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
crm_db_uri = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"


# Establish database connections
shipment_db = SQLDatabase.from_uri(shipment_db_uri)
crm_db = SQLDatabase.from_uri(crm_db_uri)

In [ ]:
# Initialize the Azure OpenAI language model
azure_llm = AzureChatOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_key=AZURE_OPENAI_KEY,
  api_version="2024-10-21",
  deployment_name=AZURE_OPENAI_DEPLOYMENT,
)

In [ ]:
# Query functions for each database
def query_shipment(query):
    return shipment_chain.invoke(query)

def query_crm(query):
    return crm_chain.invoke(query)

# Function to retrieve database schema information
def get_schema_info():
    from sqlalchemy import text
    with shipment_db._engine.connect() as connection:
        query = text("""
        SELECT
            cols.table_schema,
            cols.table_name,
            cols.column_name,
            cols.data_type,
            cols.is_nullable,
            cons.constraint_type,
            cons.constraint_name,
            fk.references_table AS referenced_table,
            fk.references_column AS referenced_column
        FROM information_schema.columns cols
        LEFT JOIN information_schema.key_column_usage kcu
            ON cols.table_schema = kcu.table_schema
            AND cols.table_name = kcu.table_name
            AND cols.column_name = kcu.column_name
        LEFT JOIN information_schema.table_constraints cons
            ON kcu.table_schema = cons.table_schema
            AND kcu.table_name = cons.table_name
            AND kcu.constraint_name = cons.constraint_name
        LEFT JOIN (
            SELECT
                rc.constraint_name,
                kcu.table_name AS references_table,
                kcu.column_name AS references_column
            FROM information_schema.referential_constraints rc
            JOIN information_schema.key_column_usage kcu
                ON rc.unique_constraint_name = kcu.constraint_name
        ) fk
            ON cons.constraint_name = fk.constraint_name
        WHERE cols.table_schema = 'public'
        ORDER BY cols.table_schema, cols.table_name, cols.ordinal_position;
        """)
        result = connection.execute(query)
        columns = result.keys()
        rows = result.fetchall()
        # Convert the result to a list of dictionaries
        schema_info = [dict(zip(columns, row)) for row in rows]
    return schema_info

# Function to share schema information between agents
def get_shared_schema_info():
    if schema_agent.schema_info is None:
        schema_agent.retrieve_and_store_schema()
    return schema_agent.schema_info

# Method to retrieve and store schema information
def retrieve_and_store_schema(agent):
    schema_info = get_schema_info()
    agent.schema_info = schema_info
    return "Schema information retrieved and stored."

In [ ]:
# Language model configuration with functions
llm_config = {
    "config_list": [
        {
        "model": AZURE_OPENAI_DEPLOYMENT,
        "temperature": 0.7,
        "api_key": AZURE_OPENAI_KEY,
        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
        "api_type": "azure",
        "api_version": "2024-10-21"
        }
    ],
    "seed": 42,
    "functions": [
        {
            "name": "query_shipment",
            "description": "Queries the Shipment database based on the provided query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The SQL query to execute on the shipment database"}
                },
                "required": ["query"]
            }
        },
        {
            "name": "query_crm",
            "description": "Queries the CRM database based on the provided query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The SQL query to execute on the CRM database"}
                },
                "required": ["query"]
            }
        },
        {
            "name": "get_schema_info",
            "description": "Retrieves the database schema and referential integrity information.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        },
        {
            "name": "get_shared_schema_info",
            "description": "Provides the stored schema information to other agents.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    ]
}

# Initialize the database chains
shipment_chain = SQLDatabaseChain(llm=azure_llm, database=shipment_db, verbose=True)
crm_chain = SQLDatabaseChain(llm=azure_llm, database=crm_db, verbose=True)

# Create assistant agents
shipment_agent = autogen.AssistantAgent(
    name="ShipmentAgent",
    llm_config=llm_config,
    description="Handles shipments in the main database.",
    system_message=(
        "Your role is to query the main database using 'query_shipment'. "
        "Focus on the shipments tables and ensure that all shipments are tracked correctly. You can make SELECT, INSERT, DELETE, UPDATE using PostgreSQL queries. Before making Update, Insert, Delete, confirm with user_proxy agents."
        "Use 'get_shared_schema_info' to retrieve schema information."
        "If the user's question has already been answered, do not provide redundant information."
    )
)

crm_agent = autogen.AssistantAgent(
    name="CRMAgent",
    llm_config=llm_config,
    description="Manages customer and product information in the second database.",
    system_message=(
        "Your role is to query the second database using 'query_crm'. "
        "Focus on maintaining the customers and product tables. You can make SELECT, INSERT, DELETE, UPDATE using PostgreSQL queries. Before making Update, Insert, Delete, confirm with user_proxy agents."
        "Use 'get_shared_schema_info' to retrieve schema information."
        "If the user's question has already been answered, do not provide redundant information."
    )
)

schema_agent = autogen.AssistantAgent(
    name="SchemaAgent",
    llm_config=llm_config,
    description="Understands and shares database schema information.",
    system_message=(
        "Your role is to retrieve and understand the database schema and referential integrity constraints. "
        "Use 'get_schema_info' to retrieve schema information and store it. "
        "If the user's question has already been answered, do not provide redundant information."
    )
)

# Register functions with the agents
shipment_agent.register_function(function_map={"query_shipment": query_shipment})
crm_agent.register_function(function_map={"query_crm": query_crm})
schema_agent.register_function(
    function_map={
        "get_schema_info": get_schema_info,
        "get_shared_schema_info": get_shared_schema_info,
    }
)

# Add schema_info attribute and bind method to schema_agent
import types
schema_agent.schema_info = None
schema_agent.retrieve_and_store_schema = types.MethodType(retrieve_and_store_schema, schema_agent)

In [ ]:
# Create a user proxy agent
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 4,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    human_input_mode="NEVER",
)

# Set up the group chat and manager
groupchat = autogen.GroupChat(
    agents=[user_proxy, schema_agent, shipment_agent, crm_agent],
    messages=[],
    max_round=20 # Maximum number of rounds in the conversation
)

manager = autogen.GroupChatManager(groupchat=groupchat)

In [ ]:
import asyncio

# Initialize chat history
chat_history = []

async def process_user_input(user_message, chat_history):
    # Append your message with role 'assistant' (to appear on the left)
    chat_history.append({"role": "assistant", "content": user_message})

    # Append a placeholder for the agent's response with role 'user' (to appear on the right)
    placeholder_index = len(chat_history)
    chat_history.append({"role": "user", "content": "Processing..."})

    # Return the updated chat history immediately
    yield chat_history, chat_history

    # Now process the agent's response
    # Use the user_proxy agent to process the message
    await asyncio.to_thread(user_proxy.initiate_chat, manager, message=user_message)

    # Collect messages from the agents
    agent_messages = [
        msg for msg in manager.groupchat.messages if msg.get("role", "") != "System"
    ]

    # Remove the placeholder
    if placeholder_index < len(chat_history):
        chat_history.pop(placeholder_index)

    # Append each agent's message to the chat history
    for msg in agent_messages:
        name = msg.get("name", "Agent")
        content = msg.get("content", "")
        role = "user"  # Agents' messages will appear on the right

        # Include the agent's name in the content
        content_with_name = f"**{name}**: {content}"

        # Append to chat history
        chat_history.append({"role": role, "content": content_with_name})

        # Yield after each agent's message to update the UI
        yield chat_history, chat_history

    # Return the final chat history
    yield chat_history, chat_history

def gradio_chat_interface():
    with gr.Blocks() as demo:
        chat_history_state = gr.State([])

        gr.Markdown("# Multi-Agent Chat Interface")

        with gr.Row():
            chatbot = gr.Chatbot(type="messages")  # Use 'messages' format
        with gr.Row():
            user_input = gr.Textbox(
                placeholder="Type your message here...",
                show_label=False
            )
            send_button = gr.Button("Send")
            clear_button = gr.Button("Clear Chat")

        async def on_user_message(user_message, chat_history):
            if user_message:
                # Process the user input and get the updated chat history
                # Use a generator to handle incremental updates
                response = process_user_input(user_message, chat_history)
                async for chat_history_update, chat_history_state_update in response:
                    await asyncio.sleep(0)
                    yield gr.update(value=chat_history_update), chat_history_state_update

        send_button.click(
            on_user_message,
            inputs=[user_input, chat_history_state],
            outputs=[chatbot, chat_history_state]
        )

        user_input.submit(
            on_user_message,
            inputs=[user_input, chat_history_state],
            outputs=[chatbot, chat_history_state]
        )

        clear_button.click(
            lambda: ([], []),
            None,
            [chatbot, chat_history_state],
            queue=False
        )

    return demo

In [ ]:
user_proxy.initiate_chat(manager, message="Which products with names are currently tracking in transit?") 

In [ ]:
user_proxy.initiate_chat(
    manager,
    message=(
        "Is Marc a Customer?"
    )
)

In [ ]:
user_proxy.initiate_chat(
    manager,
    message=(
        "Can you add MarcR with email address marc@contoso.com, phone number +1 123 456 7890 and address in 1 Main Street, Bellevue?"
    )
)

In [ ]:
user_proxy.initiate_chat(
    manager,
    message=(
        "Can you create a new shipment of 1 Laptop and 1 Smartphone to MarcR and ensure shipment is updated to Departed Origin from the location in New York and towards Los Angeles date is today?"
    )
)

In [ ]:
# Run the Gradio interface
demo = gradio_chat_interface()
demo.launch()